In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, LayerNormalization, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from numpy.random import seed

In [2]:
Z_eff = pd.read_csv('data_Z_eff.csv',header=None)
R_eff = pd.read_csv('data_R_eff.csv',header = None)
R_org_eff = pd.read_csv('data_R_org_eff.csv',header = None)
group_ind = pd.read_csv('data_group_ind.csv',header =None)

In [3]:
# time VS. feature
display(Z_eff)

# return rate
# truncated from R-org_eff. used for training and validation
display(R_eff)

# real-world data
display(R_org_eff)

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.605691,0.183234,-0.733577,-0.452301,0.693095,0.766010,0.934426,0.000000,-0.602862,0.885659,...,-0.660839,-0.442308,0.919580,1.000000,0.000000,-0.497942,-0.464286,0.290173,0.000000,-0.008000
1,-0.087398,0.007186,0.251825,-0.005612,-0.148338,0.008621,0.002342,0.000000,-0.617174,0.844961,...,-0.099650,-0.795455,0.673077,0.268591,0.000000,-0.171468,-0.250000,-0.003468,0.000000,-0.008000
2,0.508130,-0.147305,0.251825,-0.095398,0.941176,-0.307882,-0.269321,0.000000,-0.577818,-0.294574,...,-0.833916,-0.026224,-0.277972,0.707787,0.000000,-0.410151,0.150510,-0.193064,0.000000,-0.178667
3,-0.930894,-0.822754,0.251825,-0.696970,0.501279,-0.906404,-0.655738,0.000000,-0.937388,-0.193798,...,-0.622378,0.454545,-0.601399,0.055118,0.000000,-0.883402,-0.250000,-0.787283,0.000000,-0.008000
4,-0.193708,-0.547305,0.181212,-0.167228,0.961637,-0.167488,0.042155,0.000000,-0.568873,0.286822,...,-0.959790,-0.026224,-0.277972,0.707787,0.000000,-0.670782,-0.214286,-0.412717,0.000000,-0.178667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2219311,0.288483,0.001147,-0.123140,0.452809,0.864931,0.017188,-0.000822,-0.222944,0.729479,0.004637,...,0.094673,0.938588,1.000000,1.000000,0.851568,0.785146,-0.248915,0.004034,0.839611,0.592838
2219312,0.244607,-0.191131,-0.123140,-0.378652,0.099268,-0.528427,-0.251130,0.487013,-0.051392,-0.511804,...,0.580590,0.938588,1.000000,1.000000,0.491289,-0.527188,-0.248915,0.122846,0.700284,-0.953289
2219313,-0.101280,0.001147,-0.123140,0.267416,0.112286,0.017188,-0.000822,0.570707,0.518915,-0.890388,...,-0.493044,1.000000,0.070307,-0.361775,0.020209,-0.327586,-0.687410,-0.052439,-0.982179,0.824056
2219314,0.329433,0.744648,0.349794,0.501685,0.084622,0.195240,-0.476367,-0.398990,-0.118487,-0.561551,...,0.417034,0.035142,0.236177,1.000000,0.491289,0.218833,0.156295,0.287862,0.594978,-0.745426


,0
0,-0.045149
1,-0.094038
2,-0.016267
3,0.000012
4,0.023208
...,...
2219311,0.091533
2219312,0.142735
2219313,0.109201
2219314,0.075775


,0
0,-0.045149
1,-0.094038
2,-0.016267
3,0.000012
4,0.023208
...,...
2219311,0.091533
2219312,0.142735
2219313,0.109201
2219314,0.075775


In [4]:
# monthly record, total 690 months
# 0~1137：t0 (m0)，0~1137 stock active
# 1138~2280：t1 (m1)，1138~2280 stock active
display(group_ind)

,0
0,0
1,1138
2,2281
3,3425
4,4571
...,...
686,2207726
687,2210541
688,2213366
689,2216313


In [5]:
# training 120月(10年)、valid 60月(5年)、test 12月(1年) 為一循環，去跑這690個月(約58年)的data。
# 第一次從第一年開始，跑16年，第二次從第二年開始，跑16年，以此類推...
# 總共跑42次循環

t_train_start = list(range(0,41*12+1,12))
t_train_end =[x+120 for x in t_train_start]
t_val_start= [x for x in t_train_end]
t_val_end = [x+60 for x in t_val_start]
t_test_start = [x for x in t_val_end]
t_test_end = [x+12 for x in t_test_start]
t_test_end[41]=678

In [6]:
train_start = group_ind.iloc[t_train_start][0].to_list()
train_end = group_ind.iloc[t_train_end][0].to_list()
test_start = group_ind.iloc[t_test_start][0].to_list()
test_end = group_ind.iloc[t_test_end][0].to_list()
val_start = group_ind.iloc[t_val_start][0].to_list()
val_end = group_ind.iloc[t_val_end][0].to_list()                         

In [7]:
train_start

[0,
 13982,
 29160,
 45643,
 63036,
 82747,
 104359,
 126950,
 149730,
 173744,
 205530,
 238201,
 266213,
 297905,
 331270,
 366436,
 403001,
 439798,
 479825,
 519039,
 560151,
 606230,
 649583,
 693694,
 740493,
 785023,
 827350,
 868091,
 905534,
 948816,
 997587,
 1054105,
 1112321,
 1176013,
 1242410,
 1309280,
 1368589,
 1427683,
 1478335,
 1523159,
 1563174,
 1608972]

In [8]:
def portf_ret(preds, y_org):
    list_buy = [idx for idx, val in preds if val > np.percentile(preds,90)]
    list_buy = [idx for idx, val in preds if val < np.percentile(preds,10)]
    portf_ret_t = np.mean(y_true_t[list_buy]) - np.mean(y_true_t[list_sell])
    return portf_ret_t

In [9]:
y_pred = []
y_pred_val = []
y_pred_test = []
loss = []
model_list = []
y_org = []
y_org_val = []
y_org_test = []
y_reff = []
y_reff_val = []
y_reff_test = []
portf_ret_nn_test = []
R_pred=R_org_eff
seed(1)

In [28]:
for retrain_idx in range(42):
    X = Z_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    X_idx = X.apply(pd.Series.nunique) != 1
    X = X.loc[:,X_idx]
    X_val = Z_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    X_test = Z_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    X_val = X_val.loc[:,X_idx]
    X_test = X_test.loc[:,X_idx]
    y =R_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    y_val = R_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    y_test = R_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:] #since it is time series data
    yorg = R_org_eff.iloc[train_start[retrain_idx]:train_end[retrain_idx],:]
    yorg_val = R_org_eff.iloc[val_start[retrain_idx]:val_end[retrain_idx],:]
    yorg_test=R_org_eff.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]
    
    model = Sequential()
    model.add(LayerNormalization())
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(256,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(128,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(32,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(16,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(8,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(4,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(2,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(1))
    model.compile(optimizer='adamax',loss='mse')
    
    early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(x=X,y=y,validation_data=(X_val,y_val), batch_size=256, epochs=50,verbose=1,callbacks=[early_stop])
    preds = model.predict(X_test).flatten()
    y_org_test = yorg_test[0].to_list()
    R_pred.iloc[test_start[retrain_idx]:test_end[retrain_idx],:]=pd.DataFrame(preds)
    
    for t in range(t_test_start[retrain_idx], t_test_end[retrain_idx]):
        month_start = group_ind.iloc[t][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        month_end = group_ind.iloc[t+1][0] - group_ind.iloc[t_test_start[retrain_idx]][0]
        preds_month = preds[month_start:month_end]
        yorg_month = y_org_test[month_start:month_end]
        list_buy = [i for i, val in enumerate(preds_month) if val > np.quantile(preds_month,.9)]
        list_sell = [i for i, val in enumerate(preds_month) if val < np.quantile(preds_month,.1)]
        print(t,month_start,month_end,len(preds_month),len(list_buy),len(list_sell))
        portf_ret_nn_test.append(np.mean([yorg_month[i] for i in list_buy]) - np.mean([yorg_month[i] for i in list_sell]))
    loss.append(min(history.history['val_loss']))
    model_list.append(model)

Epoch 1/50
803/803 [==============================] - 21s 24ms/step - loss: 0.0315 - val_loss: 0.0112
Epoch 2/50
803/803 [==============================] - 19s 24ms/step - loss: 0.0090 - val_loss: 0.0096
Epoch 3/50
803/803 [==============================] - 20s 25ms/step - loss: 0.0086 - val_loss: 0.0096
Epoch 4/50
803/803 [==============================] - 22s 27ms/step - loss: 0.0086 - val_loss: 0.0096
Epoch 5/50
803/803 [==============================] - 22s 28ms/step - loss: 0.0085 - val_loss: 0.0097
Epoch 6/50
803/803 [==============================] - 22s 27ms/step - loss: 0.0085 - val_loss: 0.0097
Epoch 7/50
803/803 [==============================] - 22s 27ms/step - loss: 0.0084 - val_loss: 0.0095
Epoch 8/50
803/803 [==============================] - 22s 27ms/step - loss: 0.0083 - val_loss: 0.0099
Epoch 9/50
803/803 [==============================] - 22s 27ms/step - loss: 0.0082 - val_loss: 0.0097
Epoch 10/50
803/803 [==============================] - 22s 27ms/step - loss: 0.008

C:\Users\lilian\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\lilian\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


229 3101 6228 3127 0 313
230 6228 9362 3134 1 314
231 9362 12604 3242 0 325
232 12604 15947 3343 0 335
233 15947 19322 3375 0 338
234 19322 22772 3450 0 345
235 22772 26292 3520 0 352
236 26292 29864 3572 1 358
237 29864 33507 3643 1 365
238 33507 37261 3754 0 376
239 37261 41112 3851 2 385
Epoch 1/50
1109/1109 [==============================] - 31s 27ms/step - loss: 0.1949 - val_loss: 0.0133
Epoch 2/50
1109/1109 [==============================] - 31s 28ms/step - loss: 0.0106 - val_loss: 0.0109
Epoch 3/50
1109/1109 [==============================] - 33s 30ms/step - loss: 0.0097 - val_loss: 0.0103
Epoch 4/50
1109/1109 [==============================] - 38s 34ms/step - loss: 0.0094 - val_loss: 0.0102
Epoch 5/50
1109/1109 [==============================] - 46s 42ms/step - loss: 0.0093 - val_loss: 0.0101
Epoch 6/50
1109/1109 [==============================] - 29s 26ms/step - loss: 0.0092 - val_loss: 0.0101
Epoch 7/50
1109/1109 [==============================] - 30s 27ms/step - loss: 0.0091

290 8256 12398 4142 415 415
291 12398 15988 3590 359 359
292 15988 19420 3432 316 344
293 19420 22873 3453 346 346
294 22873 26404 3531 353 353
295 26404 30000 3596 360 360
296 30000 33650 3650 365 365
297 33650 37287 3637 364 364
298 37287 40878 3591 359 359
299 40878 44530 3652 366 366
Epoch 1/50
1386/1386 [==============================] - 44s 30ms/step - loss: 0.1266 - val_loss: 0.0167
Epoch 2/50
1386/1386 [==============================] - 47s 34ms/step - loss: 0.0123 - val_loss: 0.0105
Epoch 3/50
1386/1386 [==============================] - 48s 35ms/step - loss: 0.0103 - val_loss: 0.0102
Epoch 4/50
1386/1386 [==============================] - 37s 27ms/step - loss: 0.0101 - val_loss: 0.0101
Epoch 5/50
1386/1386 [==============================] - 38s 28ms/step - loss: 0.0099 - val_loss: 0.0096
Epoch 6/50
1386/1386 [==============================] - 40s 29ms/step - loss: 0.0098 - val_loss: 0.0096
Epoch 7/50
1386/1386 [==============================] - 43s 31ms/step - loss: 0.0096 - 

C:\Users\lilian\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\lilian\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


...

2048/2048 [==============================] - 95s 46ms/step - loss: 0.0093 - val_loss: 0.0136
Epoch 8/50
2048/2048 [==============================] - 55s 27ms/step - loss: 0.0092 - val_loss: 0.0135
Epoch 9/50
2048/2048 [==============================] - 62s 30ms/step - loss: 0.0092 - val_loss: 0.0135
Epoch 10/50
2048/2048 [==============================] - 91s 44ms/step - loss: 0.0091 - val_loss: 0.0133
Epoch 11/50
2048/2048 [==============================] - 81s 40ms/step - loss: 0.0090 - val_loss: 0.0133
Epoch 12/50
2048/2048 [==============================] - 56s 27ms/step - loss: 0.0090 - val_loss: 0.0133
Epoch 13/50
2048/2048 [==============================] - 65s 32ms/step - loss: 0.0090 - val_loss: 0.0133
Epoch 14/50
2048/2048 [==============================] - 105s 51ms/step - loss: 0.0089 - val_loss: 0.0133
Epoch 15/50
2048/2048 [==============================] - 59s 29ms/step - loss: 0.0089 - val_loss: 0.0133
Epoch 16/50
2048/2048 [==============================] - 57s 28ms/st

Epoch 1/50
2210/2210 [==============================] - 63s 28ms/step - loss: 0.0930 - val_loss: 0.0102
Epoch 2/50
2210/2210 [==============================] - 88s 40ms/step - loss: 0.0119 - val_loss: 0.0085
Epoch 3/50
2210/2210 [==============================] - 85s 39ms/step - loss: 0.0113 - val_loss: 0.0083
Epoch 4/50
2210/2210 [==============================] - 58s 26ms/step - loss: 0.0112 - val_loss: 0.0083
Epoch 5/50
2210/2210 [==============================] - 66s 30ms/step - loss: 0.0111 - val_loss: 0.0083
Epoch 6/50
2210/2210 [==============================] - 108s 49ms/step - loss: 0.0111 - val_loss: 0.0083
Epoch 7/50
2210/2210 [==============================] - 60s 27ms/step - loss: 0.0110 - val_loss: 0.0083
Epoch 8/50
2210/2210 [==============================] - 61s 28ms/step - loss: 0.0110 - val_loss: 0.0083
Epoch 9/50
2210/2210 [==============================] - 89s 40ms/step - loss: 0.0110 - val_loss: 0.0083
Epoch 10/50
2210/2210 [==============================] - 87s 39

1773/1773 [==============================] - 66s 37ms/step - loss: 0.0108 - val_loss: 0.0083
Epoch 5/50
1773/1773 [==============================] - 94s 53ms/step - loss: 0.0108 - val_loss: 0.0084
Epoch 6/50
1773/1773 [==============================] - 46s 26ms/step - loss: 0.0107 - val_loss: 0.0084
Epoch 7/50
1773/1773 [==============================] - 55s 31ms/step - loss: 0.0107 - val_loss: 0.0084
Epoch 8/50
1773/1773 [==============================] - 81s 45ms/step - loss: 0.0107 - val_loss: 0.0083
Epoch 9/50
1773/1773 [==============================] - 71s 40ms/step - loss: 0.0107 - val_loss: 0.0084
Epoch 00009: early stopping
612 0 3057 3057 306 306
613 3057 6135 3078 308 308
614 6135 9176 3041 304 304
615 9176 12218 3042 305 305
616 12218 15265 3047 305 305
617 15265 18322 3057 306 306
618 18322 21358 3036 304 304
619 21358 24406 3048 305 305
620 24406 27440 3034 304 304
621 27440 30483 3043 305 305
622 30483 33531 3048 305 305
623 33531 36584 3053 306 306
Epoch 1/50
1677/1677 

C:\Users\lilian\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\lilian\anaconda3\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


637 2887 5775 2888 165 0
638 5775 8679 2904 193 0
639 8679 11544 2865 180 0
640 11544 14422 2878 196 0
641 14422 17292 2870 179 0
642 17292 20159 2867 167 0
643 20159 23018 2859 158 0
644 23018 25872 2854 159 0
645 25872 28723 2851 167 0
646 28723 31556 2833 170 0
647 31556 34414 2858 171 0
Epoch 1/50
1573/1573 [==============================] - 51s 31ms/step - loss: 0.0727 - val_loss: 0.0086
Epoch 2/50
1573/1573 [==============================] - 43s 28ms/step - loss: 0.0099 - val_loss: 0.0080
Epoch 3/50
1573/1573 [==============================] - 48s 30ms/step - loss: 0.0096 - val_loss: 0.0078
Epoch 4/50
1573/1573 [==============================] - 63s 40ms/step - loss: 0.0094 - val_loss: 0.0077
Epoch 5/50
1573/1573 [==============================] - 92s 59ms/step - loss: 0.0094 - val_loss: 0.0078
Epoch 6/50
1573/1573 [==============================] - 41s 26ms/step - loss: 0.0093 - val_loss: 0.0077
Epoch 7/50
1573/1573 [==============================] - 43s 28ms/step - loss: 0.0093

In [29]:
np.nanmean(portf_ret_nn_test)/np.nanstd(portf_ret_nn_test)*np.sqrt(12)

8.52207574489709